In [2]:
from openpyxl import load_workbook
import win32com.client as win32
from openpyxl.utils import get_column_letter   # solo para convertir números a letras
import os 


In [3]:
def Archivo2Work():
    directorio =  os.getcwd() + '\Inventario'
    archivos_excel = []
    countador = 0
    for archivo in os.listdir(directorio):
        if archivo.endswith(".xlsx") and archivo.startswith("exist"):
            archivos_excel.append(archivo)
            ArchivoPAraWork =archivos_excel[countador]
        countador += 1
    if (countador != 0):
        return ArchivoPAraWork, directorio, True
    return "Nada", directorio, False

In [4]:
def formto_Existenciaas2(Archivoo):
    wb = load_workbook(Archivoo)
    ws = wb['EXISTENCIASVALORIZADAS']
    # Quitar agrupaciones (outline) en todas las columnas                                                                                                                                                                                            

    # Descombinar un rango específico
    ws.unmerge_cells("A1:K1")
    ws.unmerge_cells("A2:H2")
    ws.unmerge_cells("I2:k2")
    ws.unmerge_cells("A3:K3")

    # 1. Eliminar filas 1 a 3
    ws.delete_rows(1, 3)   # (fila_inicial, cantidad)
    # 2. Eliminar la columna K (columna 11)
    ws.delete_cols(11)

    # Guardar cambios
    wb.save(Archivoo)
                                                                                                                                                                                                                                                                                                                                                                                                 
                                                                                                                                                                                                                                                                                                                                                                                                 

In [5]:
def formto_Existenciaas3(Archivoo):
    wb = load_workbook(Archivoo)
    ws = wb['EXISTENCIASVALORIZADAS']

    for row in ws.row_dimensions.values():
        row.outlineLevel = 0


    # Quitar agrupaciones (outline) en todas las columnas                                                                                                                                                                                            
    for col in ws.column_dimensions.values():
        col.outlineLevel = 0

        # 3. (Opcional) No es necesario 'seleccionar' A1 para guardar
        #    pero si quieres asegurarte de que A1 sea la celda activa:
        ws.sheet_view.selection[0].activeCell = "A1"
        ws.sheet_view.selection[0].sqref = "A1"

        # --- Calcular filas dinámicas ---
        fila_ini = 4
        # Última fila usada (puede variar según tus datos)
        ultima_fila = ws.max_row   

        # Cambiar alto de todas las filas del rango
        for fila in range(fila_ini, ultima_fila + 1):
            ws.row_dimensions[fila].height = 15
            ws.row_dimensions[fila].hidden = False  # Asegura que esté visible
        # Guardar cambios
        wb.save(Archivoo)

        # Recorrer desde la última fila hacia arriba
        for fila in range(ws.max_row, 0, -1):
            celda_a = ws[f"A{fila}"].value
            celda_b = ws[f"B{fila}"].value

            # Si A o B está vacía, eliminar la fila
            if celda_a is None or celda_b is None:
                ws.delete_rows(fila)

        # Guardar cambios
        wb.save(Archivoo)

In [6]:
FilePAraWork,directorioop, estadoos = Archivo2Work()
carpeta_old = os.path.join( directorioop,FilePAraWork)
print(carpeta_old)

c:\Proyecto_COPEC\Excel_w_Python_Project_Existencias\Inventario\existenciasvalorizadas_20251009094925.xlsx


A patir de Aqui con las librerias de Excel

In [7]:
#formto_Existenciaas(carpeta_old)
formto_Existenciaas2(carpeta_old)



In [8]:
formto_Existenciaas3(carpeta_old)

Envio de Correo

In [15]:
import smtplib, os
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

def enviar_correo(asunto, Texto_Enciado, destinatario, archivos=None, cc=None, bcc=None):
    remitente = "pcaliman@gmail.com"
    clave = "glhz qhos bbww sgyc"
    servidor_smtp = "smtp.gmail.com"
    puerto = 587

    # Normalizar destinatarios a listas
    def _tolist(x):
        if not x:
            return []
        return x if isinstance(x, (list, tuple, set)) else [x]

    to_list  = _tolist(destinatario)
    cc_list  = _tolist(cc)
    bcc_list = _tolist(bcc)

    # Cabeceras (To/CC visibles). BCC nunca va en cabeceras.
    mensaje = MIMEMultipart()
    mensaje["From"] = remitente
    mensaje["To"] = ", ".join(to_list)
    if cc_list:
        mensaje["Cc"] = ", ".join(cc_list)
    mensaje["Subject"] = asunto
    mensaje.attach(MIMEText(Texto_Enciado, "html"))

    # Adjuntos (acepta str o lista). Verifica que sea archivo.
    if archivos:
        if isinstance(archivos, str):
            archivos = [archivos]
        for ruta in archivos:
            if os.path.isfile(ruta):
                with open(ruta, "rb") as f:
                    parte = MIMEBase("application", "octet-stream")
                    parte.set_payload(f.read())
                encoders.encode_base64(parte)
                parte.add_header("Content-Disposition", f"attachment; filename={os.path.basename(ruta)}")
                mensaje.attach(parte)
            else:
                print(f"⚠️ No es un archivo válido (o no existe): {ruta}")

    # Lista real de envío para SMTP = To + CC + BCC
    all_rcpts = list(set(to_list + cc_list + bcc_list))

    try:
        with smtplib.SMTP(servidor_smtp, puerto) as servidor:
            servidor.starttls()
            servidor.login(remitente, clave)
            servidor.sendmail(remitente, all_rcpts, mensaje.as_string())
        print("✅ Correo enviado correctamente.")
    except Exception as e:
        print("❌ Error al enviar correo:", e)


In [9]:
print(carpeta_old)


c:\Proyecto_COPEC\Excel_w_Python_Project_Existencias\Inventario\existenciasvalorizadas_20251009094925.xlsx


In [16]:
enviar_correo(
    asunto="Existencias Valorizadas",
    Texto_Enciado="<h3>Adjunto el reporte de Existencias Valorizadas</h3>",
    destinatario=["pcaliman@hotmail.com", "20110@escopec.cl"],
    archivos=[carpeta_old]
)


✅ Correo enviado correctamente.
